In [3]:
import torch
import numpy as np 
from datautil import *
from math import log2
train_set, test_set = read_data_as_np(None)
NUM_FEATURES = 3501
DATASET_SIZE = len(train_set)
feature_occ = [0] * NUM_FEATURES
num_p_s = [0] * NUM_FEATURES
num_n_s = [0] * NUM_FEATURES
num_p = 0
num_n = 0
print(f"Loaded data : Matrix size {len(train_set)} x {len(train_set[0][0])}")
print(f"Loaded data : Matrix size {len(test_set)} x {len(test_set[0][0])}")

Loaded data : Matrix size 801 x 3501
Loaded data : Matrix size 201 x 3501


In [4]:
for i in range(DATASET_SIZE):
    fp, tox = train_set[i]
    if tox == 1: num_p += 1 
    else: num_n += 1
    for j in range(NUM_FEATURES):
        if fp[j]: 
            feature_occ[j] += 1
            if tox == 1: num_p_s[j] += 1 
            else: num_n_s[j] += 1

In [53]:
np.set_printoptions(threshold=sys.maxsize)

weight = [0] * NUM_FEATURES
for i in range(NUM_FEATURES):
    weight[i] = abs(log2((num_p_s[i] + .1) / (num_n_s[i] + .1)))
    
weight = np.array(weight)
print((weight))

[0.60150265 0.55197178 0.         0.         0.         0.
 3.45943162 0.         0.         0.60139892 0.61085281 0.60936973
 0.62644821 0.89602158 0.57267681 0.78815091 0.69427096 1.28010792
 0.61693911 0.62187407 0.598881   0.         3.45943162 1.44771073
 1.21556308 0.97654103 0.         0.         0.         0.
 0.         0.97654103 0.         1.14300562 0.77333798 3.45943162
 0.         0.96523458 0.         1.49476469 0.         0.
 0.         2.26589406 5.357552   0.         0.         0.
 3.45943162 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         3.45943162
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.  

In [54]:
import numpy as np
from sklearn import svm
from datautil import *

train_set, test_set = read_data_as_np(None)
weighted_train_set, weighted_test_set = read_data_as_np(weight)

print(f"Loaded data : Matrix size {len(train_set)} x {len(train_set[0][0])}")

Loaded data : Matrix size 801 x 3501


In [55]:
def X_Y_Split(data_points):
    X, Y = [], []
    for idx in range(len(data_points)):
        x, y = data_points[idx]
        X.append(x)
        Y.append(y)
    X = np.array(X)
    Y = np.array(Y)
    return X, Y

def evaluate_classifier(classifier, test_data, title=""):
    print(f"Evaluate {title}")
    testX, testY = X_Y_Split(test_data)
    total = testY.shape[0]
    testY = testY.ravel()
    pred = classifier.predict(testX)
    result = (pred==testY)
    correct = (np.count_nonzero(result))
    print(f"{correct}/{total} correct ({100*correct/total:.3f}%)")

tX, tY = X_Y_Split(train_set)
wtX, wtY = X_Y_Split(weighted_train_set)

In [57]:
from sklearn.svm import SVC
from sklearn.metrics.pairwise import *
from e3fp.fingerprint.metrics.array_metrics import soergel, tanimoto, dice, cosine, pearson

poly_clf = SVC(kernel='poly', degree=5, probability=False, coef0=0)
poly_clf.fit(tX, tY)
evaluate_classifier(poly_clf, test_set, "Degree 5 polynomial")

tanimoto_clf = SVC(kernel=tanimoto)
tanimoto_clf.fit(tX, tY)
evaluate_classifier(tanimoto_clf, test_set, "e3fp-tanimoto")

poly_clf = SVC(kernel='poly', degree=5, probability=False, coef0=0)
poly_clf.fit(wtX, wtY)
evaluate_classifier(poly_clf, weighted_test_set, "Degree 5 polynomial-w")

tanimoto_clf = SVC(kernel=tanimoto)
tanimoto_clf.fit(wtX, wtY)
evaluate_classifier(tanimoto_clf, weighted_test_set, "e3fp-tanimoto-w")

/home/gratus/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Evaluate Degree 5 polynomial
129/201 correct (64.179%)
Evaluate e3fp-tanimoto
135/201 correct (67.164%)


/home/gratus/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/gratus/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Evaluate Degree 5 polynomial-w
123/201 correct (61.194%)
Evaluate e3fp-tanimoto-w
130/201 correct (64.677%)


/home/gratus/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
